In [1]:
import pandas as pd

df = pd.read_csv("../Trell.csv")

# removing redundant first column from the dataframe
df = df.iloc[:, 1:]

unique_val = df["userId"].value_counts()

# no of unique users in the dataset
print(len(unique_val))


286641


In [2]:
filter_col = [col for col in df if col.startswith('Category') | col.startswith('languageId') | col.startswith('Age') | col.startswith('gender') | col.startswith('duration_preprocessedduration') | col.startswith('target_gender') | col.startswith('duration_preprocessed_modeduration') | col.startswith('duration_preprocessed_modeduration') | col.startswith('User_Interest') | col.startswith('tier') | col.startswith('event_type') | col.startswith('categoryId') ]

filter_col += ["userId", "viewing_date_global", "trail_avgtimeSpent", "timespent_global", "completion_global", "duration_preprocessed_mode_freq", "completion_mode_score"]

print(df[filter_col].head(10))

   Category_86  Category_97  Category_33  Category_28  Category_35  \
0          0.0            1            1          0.0          0.0   
1          0.0            0            0          0.0          0.0   
2          0.0            1            1          0.0          0.0   
3          0.0            1            0          0.0          0.0   
4          0.0            1            1          0.0          0.0   
5          0.0            1            1          0.0          0.0   
6          0.0            1            1          0.0          0.0   
7          0.0            1            0          0.0          0.0   
8          0.0            1            0          0.0          0.0   
9          0.0            0            0          0.0          0.0   

   Category_118  Category_68  Category_75  Category_127  Category_4  ...  \
0             0          0.0          0.0             1           1  ...   
1             0          0.0          0.0             0           1  ...   
2